In [ ]:
import config
# config.BINANCE_API_KEY
# config.SECRET_BINANCE_API_KEY
# note: change the api to only allow access from trusted apis
# ip 4.71.145.140

# 1. Setup

In [ ]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
symbol = 'BTCUSDT'

# 2. Authenticate

In [ ]:
client = Client(config.BINANCE_API_KEY, config.SECRET_BINANCE_API_KEY,{"verify": False, "timeout": 20})

# 3. Get Tickers

In [ ]:
tickers = client.get_all_tickers()
ticker_df = pd.DataFrame(tickers)
ticker_df.set_index('symbol',inplace=True)
display(tickers_df.head())

# 4. Get Depth

In [ ]:
# allows you to retrieve market depth data from binance
# ticker code can be passed thru to give max num of tickers
# depth holds diff bids and diff price points to help analyze spread
# shows bids and asks
depth = client.get_order_book(symbol = symbol)
depth_df = pd.DataFrame(depth['bids'])
depth_df.columns = ['Price', 'Volume']
display(depth_df.head())

# 5. Get Historical Data

In [ ]:
client.get_historical_klines??

In [ ]:
# get_historical_klines allows you to retrieve spot and futures OHLCV data from binance
# kline interval can be adjusted to much smaller intervals if needed
historical = client.get_historical_klines(symbol, client.KLINE_INTERTVAL_1DAY, '1 Jan 2011')
hist_df = pd.DataFrame(historical)
hist_df.columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                    'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']
display(hist_df.tail())

In [ ]:
# columns are all stored as objects / strings currently
hist_df.dtypes

# 6. Preprocess Historical Data

In [ ]:
# hist_df['Open Time'] is record in some epoch unix s time unit (look this up)
hist_df['Open Time'] = pd.to_datetime(hist_df['Open Time']/1000,unit='s')
hist_df['Close Time'] = pd.to_datetime(hist_df['Close Time']/1000,unit='s')

In [ ]:
numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume', 'Ignore']

In [ ]:
hist_df[numeric_columns] = hist_df[numeric_columns].apply(pd.to_numeric,axis=1)

In [ ]:
# columns are in appropriate dtype now
hist_df.dtypes

# 7. Visualize

In [ ]:
import mplfinance as mpf
days = 100

In [ ]:
# tail is important to be set here b/c otherwise you cannnot see the details
mpf.plot(hist_df.set_index('Close Time').tail(days))

In [ ]:
mpf.plot(hist_df.set_index('Close Time').tail(days),
         type='candle',
         style='charles',
        volume = True,
        title = f"{symbol}last {days} days",
        mav = (10,20,30))

buys when there is golden cross' (50 day moving average crosses above 200 day moving average) and sells when there is a 'death cross' (50 day moving average crosses below 200 day moving average).